In [1]:
!nvidia-smi

Thu Jul  4 00:21:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     Off | 00000000:01:00.0 Off |                   On |
| N/A   31C    P0              28W / 165W |   9270MiB / 24576MiB |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+--

In [2]:
import os

os.environ['MPLCONFIGDIR'] = '/tmp/matplotlib'
os.environ['WANDB_CACHE_DIR'] = '/tmp/wandb_cache'
os.environ['JAX_LOG_COMPILATION'] = '1'

import logging
import time
from dataclasses import dataclass
from functools import partial
from math import floor
from typing import Any, Dict, Tuple, List, Callable
import pickle
from flax import serialization
#logging.basicConfig(level=logging.DEBUG)
import hydra
from omegaconf import OmegaConf, DictConfig
import jax
import jax.numpy as jnp
from hydra.core.config_store import ConfigStore
from qdax.core.map_elites import MAPElites
from qdax.types import RNGKey, Genotype
from qdax.utils.sampling import sampling 
from qdax.core.containers.mapelites_repertoire import compute_cvt_centroids, MapElitesRepertoire
from qdax.core.neuroevolution.networks.networks import MLP, MLPRein
from qdax.core.emitters.rein_var import REINConfig, REINEmitter
#from qdax.core.emitters.rein_emitter_advanced import REINaiveConfig, REINaiveEmitter
from qdax.core.neuroevolution.buffers.buffer import QDTransition
from qdax.environments import behavior_descriptor_extractor
from qdax.tasks.brax_envs import reset_based_scoring_function_brax_envs as scoring_function
from utils import Config, get_env
from qdax.core.emitters.mutation_operators import isoline_variation
import wandb
from qdax.utils.metrics import CSVLogger, default_qd_metrics
from qdax.utils.plotting import plot_map_elites_results, plot_2d_map_elites_repertoire
import matplotlib.pyplot as plt
from set_up_brax import get_reward_offset_brax
from qdax import environments_v1, environments


2024-07-03 23:48:31.535384: W external/xla/xla/service/platform_util.cc:198] unable to create StreamExecutor for CUDA:0: failed initializing StreamExecutor for CUDA device ordinal 0: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 12549357568
CUDA backend failed to initialize: INTERNAL: no supported devices found for platform CUDA (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [4]:
def get_env(env_name):
    if env_name == "hopper_uni":
        episode_length = 1000
        
        env = environments_v1.create(env_name, episode_length=episode_length)
    elif env_name == "halfcheetah_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length)
        
    elif env_name == "walker2d_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length)	
    elif env_name == "ant_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length, use_contact_forces=False, exclude_current_positions_from_observation=True)
    elif env_name == "humanoid_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length, exclude_current_positions_from_observation=True)	
    '''
    elif env_name == "ant_omni":
        episode_length = 250
        max_bd = 30.

        env = environments.create(env_name, episode_length=episode_length, use_contact_forces=False, exclude_current_positions_from_observation=False)	
    elif env_name == "humanoid_uni":
        episode_length = 1000
        max_bd = 1.

        env = environments.create(env_name, episode_length=episode_length)	
    else:
        ValueError(f"Environment {env_name} not supported.")
    '''
    return env

In [5]:
@dataclass
class Config:
    """Configuration from this experiment script
    """
    # Env config
    #alg_name: str
    seed: int
    env_name: str
    episode_length: int
    policy_hidden_layer_sizes: Tuple[int, ...]   
    # ME config
    num_evaluations: int
    num_iterations: int
    batch_size: int
    num_samples: int
    fixed_init_state: bool
    discard_dead: bool
    # Emitter config
    iso_sigma: float
    line_sigma: float
    #crossover_percentage: float
    # Grid config 
    grid_shape: Tuple[int, ...]
    num_init_cvt_samples: int
    num_centroids: int
    # Log config
    log_period: int
    store_repertoire: bool
    store_repertoire_log_period: int
    
    # REINFORCE Parameters
    proportion_mutation_ga : float
    rollout_number: int
    num_rein_training_steps: int
    adam_optimizer: bool
    learning_rate: float
    discount_rate: float
    temperature: int
    buffer_size: int

In [6]:
config = Config(
    seed=0,
    env_name='ant_uni',
    episode_length=20,
    policy_hidden_layer_sizes=[128, 128],
    num_evaluations=0,
    num_iterations=200,
    num_samples=32,
    batch_size=10,
    fixed_init_state=False,
    discard_dead=False,
    grid_shape=[50, 50],
    num_init_cvt_samples=50000,
    num_centroids=1296,
    log_period=400,
    store_repertoire=True,
    store_repertoire_log_period=800,
    iso_sigma=0.005,
    line_sigma=0.05,
    proportion_mutation_ga=0.5,
    rollout_number=1, # Num of episodes used for gradient estimate
    num_rein_training_steps=1, # Num gradient steps per generation
    buffer_size=400, # Size of the replay buffer
    adam_optimizer=True,
    learning_rate=1e-3,
    discount_rate=0.99,
    temperature=0,
)

In [7]:
# Init a random key
random_key = jax.random.PRNGKey(config.seed)

# Init environment
env = get_env("ant_uni")
reset_fn = jax.jit(env.reset)

# Compute the centroids
centroids, random_key = compute_cvt_centroids(
    num_descriptors=env.behavior_descriptor_length,
    num_init_cvt_samples=config.num_init_cvt_samples,
    num_centroids=config.num_centroids,
    minval=0,
    maxval=1,
    random_key=random_key,
)
# Init policy network
policy_layer_sizes = config.policy_hidden_layer_sizes #+ (env.action_size,)
print(policy_layer_sizes)

'''
policy_network = MLPRein(
    action_size=env.action_size,
    layer_sizes=policy_layer_sizes,
    kernel_init=jax.nn.initializers.orthogonal(scale=jnp.sqrt(2)),
    kernel_init_final=jax.nn.initializers.orthogonal(scale=0.01),
)
'''
policy_network = MLPRein(
    action_size=env.action_size,
    layer_sizes=policy_layer_sizes,
    kernel_init=jax.nn.initializers.lecun_uniform(),
    kernel_init_final=jax.nn.initializers.lecun_uniform(),
)


# Init population of controllers

# maybe consider adding two random keys for each policy
random_key, subkey = jax.random.split(random_key)
keys = jax.random.split(subkey, num=config.batch_size)
#split_keys = jax.vmap(lambda k: jax.random.split(k, 2))(keys)
#keys1, keys2 = split_keys[:, 0], split_keys[:, 1]
fake_batch_obs = jnp.zeros(shape=(config.batch_size, env.observation_size))
init_params = jax.vmap(policy_network.init)(keys, fake_batch_obs)

param_count = sum(x[0].size for x in jax.tree_util.tree_leaves(init_params))
print("Number of parameters in policy_network: ", param_count)

# Define the fonction to play a step with the policy in the environment
def play_step_fn(env_state, policy_params, random_key):
    #random_key, subkey = jax.random.split(random_key)
    actions = policy_network.apply(policy_params, env_state.obs)
    state_desc = env_state.info["state_descriptor"]
    next_state = env.step(env_state, actions)

    transition = QDTransition(
        obs=env_state.obs,
        next_obs=next_state.obs,
        rewards=next_state.reward,
        dones=next_state.done,
        truncations=next_state.info["truncation"],
        actions=actions,
        state_desc=state_desc,
        next_state_desc=next_state.info["state_descriptor"],
        #desc=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
        #desc_prime=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
    )

    return next_state, policy_params, random_key, transition

# Prepare the scoring function
bd_extraction_fn = behavior_descriptor_extractor['ant_uni']
scoring_fn = partial(
    scoring_function,
    episode_length=env.episode_length,
    play_reset_fn=reset_fn,
    play_step_fn=play_step_fn,
    behavior_descriptor_extractor=bd_extraction_fn,
)
#reward_offset = get_reward_offset_brax(env, config.env_name)
#print(f"Reward offset: {reward_offset}")

me_scoring_fn = partial(
sampling,
scoring_fn=scoring_fn,
num_samples=config.num_samples,
)



reward_offset = 0



# Get minimum reward value to make sure qd_score are positive


# Define a metrics function
    # Define a metrics function
metrics_function = partial(
    default_qd_metrics,
    qd_offset=reward_offset * config.env.episode_length,
)

# Define the PG-emitter config

me_mcpg_config = MEMCPGConfig(
    proportion_mutation_ga=config.proportion_mutation_ga,
    no_agents=config.no_agents,
    batch_size=config.batch_size,
    mini_batch_size=config.mini_batch_size,
    no_epochs=config.no_epochs,
    buffer_size=config.buffer_size,
    learning_rate=config.learning_rate,
    adam_optimizer=config.adam_optimizer,
    clip_param=config.clip_param,
)

variation_fn = partial(
    isoline_variation, iso_sigma=config.iso_sigma, line_sigma=config.line_sigma
)

me_mcpg_emitter = MEMCPGEmitter(
    config=me_mcpg_config,
    policy_network=policy_network,
    env=env,
    variation_fn=variation_fn,
    )

'''
rein_emitter = REINaiveEmitter(
    config=rein_emitter_config,
    policy_network=policy_network,
    env=env,
    )
'''
'''
me_scoring_fn = partial(
    sampling,
    scoring_fn=scoring_fn,
    num_samples=config.num_samples,
)
'''

# Instantiate MAP Elites
map_elites = MAPElites(
    scoring_function=scoring_fn,
    emitter=me_mcpg_emitter,
    metrics_function=metrics_function,
)

# compute initial repertoire
repertoire, emitter_state, random_key = map_elites.init(init_params, centroids, random_key)

log_period = 1
num_loops = int(config.num_iterations / log_period)


# Main loop
map_elites_scan_update = map_elites.scan_update
'''
for i in range(num_loops):
    print(f"Loop {i+1}/{num_loops}")
    start_time = time.time()
    
    (repertoire, emitter_state, random_key,), current_metrics = jax.lax.scan(
        map_elites_scan_update,
        (repertoire, emitter_state, random_key),
        (),
        length=log_period,
    )
    timelapse = time.time() - start_time
    
'''


[128, 128]
Number of parameters in policy_network:  21264
10


/vol/bitbucket/km2120/QD-DRL/me-with-sample-based-drl/qdax/core/map_elites.py:82: UserWarning: This type of repertoire does not store the extra scores computed by the scoring function
  repertoire = MapElitesRepertoire.init(


AssertionError: buffer_size must be a multiple of episode_length

In [7]:
repertoire, emitter_state, metrics, random_key = map_elites.update(repertoire, emitter_state, random_key)

Length : 20
reward*mask shape: (1, 20)


In [8]:
x = emitter_state.emitter_states[0].trajectory_buffer

In [10]:
x.data

Array([[ 0.5135491 ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.5104991 ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 0.50859904,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.4783475 ,  0.9828977 ,  0.18337119, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.6665856 ,  0.9911529 , -0.00221902, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.5599    ,  0.98609835, -0.04763195, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [10]:
x.data[jnp.array(x.episodic_data[0], dtype=int)].shape

(20, 75)

In [11]:
x.episodic_data

Array([[  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
        110., 120., 130., 140., 150., 160., 170., 180., 190.],
       [  1.,  11.,  21.,  31.,  41.,  51.,  61.,  71.,  81.,  91., 101.,
        111., 121., 131., 141., 151., 161., 171., 181., 191.],
       [  2.,  12.,  22.,  32.,  42.,  52.,  62.,  72.,  82.,  92., 102.,
        112., 122., 132., 142., 152., 162., 172., 182., 192.],
       [  3.,  13.,  23.,  33.,  43.,  53.,  63.,  73.,  83.,  93., 103.,
        113., 123., 133., 143., 153., 163., 173., 183., 193.],
       [  4.,  14.,  24.,  34.,  44.,  54.,  64.,  74.,  84.,  94., 104.,
        114., 124., 134., 144., 154., 164., 174., 184., 194.],
       [  5.,  15.,  25.,  35.,  45.,  55.,  65.,  75.,  85.,  95., 105.,
        115., 125., 135., 145., 155., 165., 175., 185., 195.],
       [  6.,  16.,  26.,  36.,  46.,  56.,  66.,  76.,  86.,  96., 106.,
        116., 126., 136., 146., 156., 166., 176., 186., 196.],
       [  7.,  17.,  27.,  37.,  47.,  57

In [10]:
rng = jax.random.PRNGKey(0)

l = int(x.current_episodic_data_size)
print(type(l))

r = x.sample_with_returns(rng, 3, episodic_data_size=l, sample_traj=True)

<class 'int'>


In [35]:
la = r[0].dones.reshape(3, 10, -1)

In [36]:
la.shape

(3, 10, 1)

In [30]:
o = 1. - jnp.clip(jnp.cumsum(r[0].dones), a_min=0., a_max=1.)     

In [31]:
o

Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [19]:
r[0].obs.shape

(30, 28)

In [22]:
l = r[0].obs.reshape(3, 10, -1)

In [23]:
l.shape

(3, 10, 28)

In [10]:
flattened_trans = r[0].flatten()

In [11]:
flattened_trans.shape

(30, 75)

In [12]:
r[1]

Array([0.7246356, 0.7246356, 0.7246356, 0.7246356, 0.7246356, 0.7246356,
       0.7246356, 0.7246356, 0.7246356, 0.7246356, 6.0819583, 6.0819583,
       6.0819583, 6.0819583, 6.0819583, 6.0819583, 6.0819583, 6.0819583,
       6.0819583, 6.0819583, 1.0134742, 1.0134742, 1.0134742, 1.0134742,
       1.0134742, 1.0134742, 1.0134742, 1.0134742, 1.0134742, 1.0134742],      dtype=float32)

In [34]:
x.episodic_data

Array([[  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [9009., 9019., 9029., ...,   nan,   nan,   nan]],      dtype=float32, weak_type=True)

In [11]:
rng = jax.random.PRNGKey(10)

e = x.sample_with_returns(rng, 3)

In [18]:
flattened_trans = e[0].flatten()

In [20]:
flattened_trans.shape

(3, 75)

In [19]:
x.data.masks

AttributeError: 'ArrayImpl' object has no attribute 'masks'

In [12]:
print(f"Episode length: {x.episode_length}\n")
print(f"Environment batch size: {x.env_batch_size}\n")
print(f"Number of trajectories: {x.num_trajectories}\n")

print(f"Current position: {x.current_position}\n")
print(f"Current size: {x.current_size}\n")
print(f"Trajectory positions: {x.trajectory_positions}\n")
print(f"Timestep positions: {x.timestep_positions}\n")
print(f"Episodic data: {x.episodic_data}\n")
print(f"Current episodic data size: {int(x.current_episodic_data_size)}\n")

Episode length: 20

Environment batch size: 10

Number of trajectories: 20

Current position: 200

Current size: 200

Trajectory positions: [1 1 1 1 1 1 1 1 1 1]

Timestep positions: [0 0 0 0 0 0 0 0 0 0]

Episodic data: [[  0.  10.  20.  30.  40.  50.  60.  70.  80.  90. 100. 110. 120. 130.
  140. 150. 160. 170. 180. 190.]
 [  1.  11.  21.  31.  41.  51.  61.  71.  81.  91. 101. 111. 121. 131.
  141. 151. 161. 171. 181. 191.]
 [  2.  12.  22.  32.  42.  52.  62.  72.  82.  92. 102. 112. 122. 132.
  142. 152. 162. 172. 182. 192.]
 [  3.  13.  23.  33.  43.  53.  63.  73.  83.  93. 103. 113. 123. 133.
  143. 153. 163. 173. 183. 193.]
 [  4.  14.  24.  34.  44.  54.  64.  74.  84.  94. 104. 114. 124. 134.
  144. 154. 164. 174. 184. 194.]
 [  5.  15.  25.  35.  45.  55.  65.  75.  85.  95. 105. 115. 125. 135.
  145. 155. 165. 175. 185. 195.]
 [  6.  16.  26.  36.  46.  56.  66.  76.  86.  96. 106. 116. 126. 136.
  146. 156. 166. 176. 186. 196.]
 [  7.  17.  27.  37.  47.  57.  67.  77.  8

In [33]:
x.current_episodic_data_size

Array(10, dtype=int32)

In [13]:
repertoire, emitter_state, metrics, random_key = map_elites.update(repertoire, emitter_state, random_key)

dones : [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Flattened transitions shape: (array(10, dtype=int32), array(75, dtype=int32))
Dones: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Flattened transitions shape: (array(10, dtype=int32), array(75, dtype=int32))
Dones: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Flattened transitions shape: (array(10, dtype=int32), array(75, dtype=int32))
Dones: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Flattened 

In [15]:
x = emitter_state.emitter_states[0].trajectory_buffer

In [16]:
print(f"Episode length: {x.episode_length}\n")
print(f"Environment batch size: {x.env_batch_size}\n")
print(f"Number of trajectories: {x.num_trajectories}\n")

print(f"Current position: {x.current_position}\n")
print(f"Current size: {x.current_size}\n")
print(f"Trajectory positions: {x.trajectory_positions}\n")
print(f"Timestep positions: {x.timestep_positions}\n")
print(f"Episodic data: {x.episodic_data}\n")
print(f"Current episodic data size: {x.current_episodic_data_size}\n")

Episode length: 20

Environment batch size: 10

Number of trajectories: 20

Current position: 0

Current size: 400

Trajectory positions: [2 2 2 2 2 2 2 2 2 2]

Timestep positions: [0 0 0 0 0 0 0 0 0 0]

Episodic data: [[  0.  10.  20.  30.  40.  50.  60.  70.  80.  90. 100. 110. 120. 130.
  140. 150. 160. 170. 180. 190.]
 [  1.  11.  21.  31.  41.  51.  61.  71.  81.  91. 101. 111. 121. 131.
  141. 151. 161. 171. 181. 191.]
 [  2.  12.  22.  32.  42.  52.  62.  72.  82.  92. 102. 112. 122. 132.
  142. 152. 162. 172. 182. 192.]
 [  3.  13.  23.  33.  43.  53.  63.  73.  83.  93. 103. 113. 123. 133.
  143. 153. 163. 173. 183. 193.]
 [  4.  14.  24.  34.  44.  54.  64.  74.  84.  94. 104. 114. 124. 134.
  144. 154. 164. 174. 184. 194.]
 [  5.  15.  25.  35.  45.  55.  65.  75.  85.  95. 105. 115. 125. 135.
  145. 155. 165. 175. 185. 195.]
 [  6.  16.  26.  36.  46.  56.  66.  76.  86.  96. 106. 116. 126. 136.
  146. 156. 166. 176. 186. 196.]
 [  7.  17.  27.  37.  47.  57.  67.  77.  87.

In [37]:
x.transition.dones

Array([0.], dtype=float32)